In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
data = pd.read_csv('airline_data.csv')

In [4]:
data.head()

,iata_code,airline_name,region,functional_currency,ebit_usd,load_factor,low_cost_carrier,airline_age,num_routes,passenger_yield,ask,avg_fleet_age,fleet_size,aircraft_utilisation
0,ET,Ethiopian Airlines,Africa,ETB,1.032027e+09,0.703,N,78.0,136.0,0.107,6.449871e+10,8.80,154.0,NaN
1,KQ,Kenya Airways,Africa,KES,7.600000e+07,0.784,N,47.0,42.0,0.111,1.480400e+10,13.50,40.0,NaN
2,LA,LATAM Airlines,Latin America,USD,1.078165e+09,0.831,N,95.0,148.0,0.090,1.370000e+11,11.48,333.0,NaN
3,AV,Avianca,Latin America,USD,6.185440e+08,0.824,N,104.0,147.0,0.083,5.470600e+10,9.30,140.0,NaN
4,AD,Azul Brazilian Airlines,Latin America,BRL,5.968403e+08,0.804,Y,15.0,160.0,0.101,4.400600e+10,7.40,183.0,10.0
